In [1]:
import numpy as np

import pygsti
from pygsti.noisymodelpacks import noisymodelpack as nmp
from pygsti.objects.operation import ComposedOp, StochasticNoiseOp

In [2]:
pspec = pygsti.objects.ProcessorSpec(10, ['Gi', 'Gxpi', 'Gypi', 'Gcnot'])

In [3]:
mdl = nmp.create_pauli_stochastic_noise_model(pspec, ['Gi', ('Gi', 9), 'Gcnot'])

In [4]:
def print_stochastic_gates(mdl):
    for lbl, op in mdl.operation_blks['gates'].items():
        print(f'Gate {lbl}\n{op}')
        if isinstance(op, ComposedOp) and isinstance(op.factorops[1], StochasticNoiseOp):
            noise_op = op.factorops[1]
            rates = noise_op._params_to_rates(noise_op.params)
            print(f'Total strength: {sum(rates)}\nRates: {rates}\n\n')

print_stochastic_gates(mdl)

Gate Gcnot
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 16, num params = 15

Total strength: 0.0
Rates: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Gate Gypi
StaticStandardOp with name Gypi and evotype densitymx

Gate Gxpi
StaticStandardOp with name Gxpi and evotype densitymx

Gate Gi
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.0
Rates: [0. 0. 0.]


Gate Gi:9
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.0
Rates: [0. 0. 0.]




In [5]:
rand_state = np.random.RandomState(2021)

# Create stochastic noise model with:
# slightly noisy idle, bad idle on qubit 9, and noisy CNOT gates
# CNOT noise allows up to weight 2 (i.e. correlated) and different each qubit
# Allow random total noise and random distribution among channels
noisy_mdl = nmp.sample_noise_model(mdl,
                                   max_total_strengths={
                                        'Gi': 0.01,
                                        ('Gi', 9): 0.5,
                                        'Gcnot': 0.1},
                                   max_weight=2,
                                   rand_state=rand_state)

In [6]:
print_stochastic_gates(noisy_mdl)

Gate Gcnot
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 16, num params = 15

Total strength: 0.060597827880740474
Rates: [0.00628481 0.00119075 0.00267954 0.00854616 0.00109832 0.00153393
 0.00645241 0.00567457 0.00672137 0.00083036 0.00050194 0.00824752
 0.00528376 0.0007424  0.00480998]


Gate Gypi
StaticStandardOp with name Gypi and evotype densitymx

Gate Gxpi
StaticStandardOp with name Gxpi and evotype densitymx

Gate Gi
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.0061652470861799005
Rates: [0.0031123  0.00185446 0.0011985 ]


Gate Gi:9
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.22607262189478217
Rates

In [7]:
# Let's try with only weight 1 on CNOT and make it symmetric
noisy_mdl2 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     max_weight=1,  # from 2
     symmetric_strengths=True, # from False
     rand_state=rand_state)

In [8]:
print_stochastic_gates(noisy_mdl2)

Gate Gcnot
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 16, num params = 15

Total strength: 0.0583704016323978
Rates: [0.00764974 0.00831625 0.01321921 0.00764974 0.         0.
 0.         0.00831625 0.         0.         0.         0.01321921
 0.         0.         0.        ]


Gate Gypi
StaticStandardOp with name Gypi and evotype densitymx

Gate Gxpi
StaticStandardOp with name Gxpi and evotype densitymx

Gate Gi
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.0073222502693899095
Rates: [0.00036518 0.00355427 0.0034028 ]


Gate Gi:9
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.4140071851295825
Rates: [0.058799

In [9]:
# Let's generate a model with deterministic total strength
noisy_mdl3 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     random_total_strength=False,   # from True
     rand_state=rand_state)

In [10]:
print_stochastic_gates(noisy_mdl3)

Gate Gcnot
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 16, num params = 15

Total strength: 0.10000000000000003
Rates: [0.00908809 0.02631445 0.01779936 0.02963581 0.         0.
 0.         0.01334927 0.         0.         0.         0.00381304
 0.         0.         0.        ]


Gate Gypi
StaticStandardOp with name Gypi and evotype densitymx

Gate Gxpi
StaticStandardOp with name Gxpi and evotype densitymx

Gate Gi
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.010000000000000002
Rates: [0.00274072 0.00232574 0.00493354]


Gate Gi:9
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.5
Rates: [0.16198248 0.20348009 0

In [11]:
# Or maybe random total strength but depolarizing
noisy_mdl4 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     random_relative_strengths=False, # from True
     rand_state=rand_state)

In [12]:
print_stochastic_gates(noisy_mdl4)

Gate Gcnot
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 16, num params = 15

Total strength: 0.06368299245545493
Rates: [0.01061383 0.01061383 0.01061383 0.01061383 0.         0.
 0.         0.01061383 0.         0.         0.         0.01061383
 0.         0.         0.        ]


Gate Gypi
StaticStandardOp with name Gypi and evotype densitymx

Gate Gxpi
StaticStandardOp with name Gxpi and evotype densitymx

Gate Gi
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.0034220893702305837
Rates: [0.0011407 0.0011407 0.0011407]


Gate Gi:9
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gi and evotype densitymx
Factor 1:
Stochastic noise operation map with dim = 4, num params = 3

Total strength: 0.41189559139564286
Rates: [0.1372985